<a href="https://colab.research.google.com/github/Nicordaro/Project_MLDL/blob/master/Project_MLDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Politecnico di Torino**

**01TXFSM - Machine learning and Deep learning**

**Incremental Learning in Image Classification**

**Cordaro Nicolò - s272145**

**Di Nepi Marco - s277959**

**Falletta Alberto - s277971**


In [0]:
# !pip3 install 'torch==1.4.0'
# !pip3 install 'torchvision==0.5.0'
# !pip3 install 'Pillow-SIMD'
# !pip3 install 'tqdm'

**Imports**

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn


import torchvision
from torchvision import transforms
from torchvision.models import resnet18
from torchvision.models import resnet34
from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
from google.colab import drive

**Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

# Init at 10 because first train is on 10 classes
NUM_CLASSES = 10

# Used for the pseudorandom shuffle of the split
SEED = 42

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.01            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 10      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 1000     # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 50

**Pre-processing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # https://gist.github.com/weiaicunzai/e623931921efefd4c331622c344d8151
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))                                 
])

**Prepare Dataset**

CIFAR100 has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another.

Each of the downloaded files is a Python "pickled" object produced with cPickle.

In [26]:
# Clone github repository with data
# if os.path.isdir('./Project_MLDL'):
!rm -rf Project_MLDL
if not os.path.isdir('./CIFAR100_tError'):
  !git clone https://github.com/Nicordaro/Project_MLDL


Cloning into 'Project_MLDL'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 107 (delta 47), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (107/107), 191.23 KiB | 3.82 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [0]:
from Project_MLDL.CIFAR100_tError import CIFAR100_tError
import numpy as np
import random

DATA_DIR = './CIFAR100'

lbls = [i for i in range(0,100)]  #Array of classes integer-encoded (?)
random.seed(SEED)
random.shuffle(lbls)

def make_data_labels(lbls):       #After shuffle, take first 10 classes, and remove the first 10 from the list passed as argument
  new_labels=[]
  for el in lbls[:10]:
    new_labels.append(el)
  lbls = lbls[10:]

  return lbls, new_labels

**Network Definition**

In [0]:
net = resnet34(pretrained=False)

**Prepare Training**

In [0]:
# Loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Parameters to optimize:
parameters_to_optimize = net.parameters()

# Optimizer
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Train & Test**

In [0]:
BATCH_TO_TEST = 10
accuracies = []

# Define test dataset outside in order to increment it, instead of initializing it every cycle iteration
test_dataset = CIFAR100_tError(DATA_DIR, train=False, transform=eval_transform, download=True)
for i in range(0,BATCH_TO_TEST): #one iteration for 10 classes
  print(f'Starting training with batch {i+1}')
  lbls, new_labels = make_data_labels(lbls)
  
  # Define Train dataset
  train_dataset = CIFAR100_tError(DATA_DIR, train=True, transform=train_transform, download=True)       
  
  # Increment dataset with new labels mapped with list comprehension
  train_dataset.increment(new_labels,[j for j in range(0+i*10,10+i*10)])
  test_dataset.increment(new_labels,[j for j in range(0+i*10,10+i*10)])

  # Change number of neurons in last fully connected layer
  net.fc = nn.Linear(512, NUM_CLASSES*(i+1))

  # Define dataloader
  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

  #prepare training
  # Loss function
  criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

  # Parameters to optimize:
  parameters_to_optimize = net.parameters()

  # Optimizer
  optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  # Scheduler
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

  # training 
  # By default, everything is loaded to cpu
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

  cudnn.benchmark # Calling this optimizes runtime

  current_step = 0

  # Start iterating over the epochs
  for epoch in range(NUM_EPOCHS):
    print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
    
    # Iterate over the dataset
    for images, labels in train_dataloader:
      # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      net.train() # Sets module in training mode

      # PyTorch, by default, accumulates gradients after each backward pass
      # We need to manually set the gradients to zero before starting a new iteration
      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = net(images)

      # Compute loss based on output and ground truth
      loss = criterion(outputs, labels)
      
      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss {}'.format(current_step, loss.item()))

      # Compute gradients for each layer and update weights
      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients

      current_step += 1

    # Step the scheduler
    scheduler.step()

  #test phase
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in tqdm(test_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    #Debugging purpose, print labels of predictions
    ##print(preds)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(test_dataset))

  #Store accuracies for plotting purposes
  accuracies.append(accuracy)

  print('Test Accuracy: {}'.format(accuracy))

Files already downloaded and verified
Starting training with batch 1
Files already downloaded and verified
Starting epoch 1/10, LR = [0.01]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss 2.5046603679656982
Starting epoch 2/10, LR = [0.01]
Step 50, Loss 1.9012675285339355
Starting epoch 3/10, LR = [0.01]
Step 100, Loss 1.5411834716796875
Starting epoch 4/10, LR = [0.01]
Step 150, Loss 1.1983280181884766
Starting epoch 5/10, LR = [0.01]
Starting epoch 6/10, LR = [0.01]
Step 200, Loss 1.135864496231079
Starting epoch 7/10, LR = [0.01]
Step 250, Loss 0.8120095133781433
Starting epoch 8/10, LR = [0.01]
Step 300, Loss 0.8160781264305115
Starting epoch 9/10, LR = [0.01]
Step 350, Loss 0.996483564376831
Starting epoch 10/10, LR = [0.01]


100%|██████████| 8/8 [00:04<00:00,  1.91it/s]


Test Accuracy: 0.57
Starting training with batch 2
Files already downloaded and verified
Starting epoch 1/10, LR = [0.01]
Step 0, Loss 3.317263126373291
Starting epoch 2/10, LR = [0.01]
Step 50, Loss 1.1504727602005005
Starting epoch 3/10, LR = [0.01]
Step 100, Loss 0.8486161231994629
Starting epoch 4/10, LR = [0.01]
Step 150, Loss 0.49848711490631104
Starting epoch 5/10, LR = [0.01]
Starting epoch 6/10, LR = [0.01]
Step 200, Loss 0.3385623097419739
Starting epoch 7/10, LR = [0.01]
Step 250, Loss 0.19625917077064514
Starting epoch 8/10, LR = [0.01]
Step 300, Loss 0.24378496408462524
Starting epoch 9/10, LR = [0.01]
Step 350, Loss 0.2602942883968353
Starting epoch 10/10, LR = [0.01]


100%|██████████| 16/16 [00:07<00:00,  2.18it/s]


Test Accuracy: 0.31
Starting training with batch 3
Files already downloaded and verified
Starting epoch 1/10, LR = [0.01]
Step 0, Loss 3.603761672973633
Starting epoch 2/10, LR = [0.01]
Step 50, Loss 0.7583374977111816
Starting epoch 3/10, LR = [0.01]
Step 100, Loss 0.49484866857528687
Starting epoch 4/10, LR = [0.01]
Step 150, Loss 0.27287012338638306
Starting epoch 5/10, LR = [0.01]
Starting epoch 6/10, LR = [0.01]
Step 200, Loss 0.028249897062778473
Starting epoch 7/10, LR = [0.01]
Step 250, Loss 0.0389537513256073
Starting epoch 8/10, LR = [0.01]
Step 300, Loss 0.0037180185317993164
Starting epoch 9/10, LR = [0.01]
Step 350, Loss 0.002068467438220978
Starting epoch 10/10, LR = [0.01]


100%|██████████| 24/24 [00:10<00:00,  2.36it/s]


Test Accuracy: 0.24466666666666667
Starting training with batch 4
Files already downloaded and verified
Starting epoch 1/10, LR = [0.01]
Step 0, Loss 3.7168548107147217
Starting epoch 2/10, LR = [0.01]
Step 50, Loss 0.9036841988563538
Starting epoch 3/10, LR = [0.01]
Step 100, Loss 0.2598949372768402
Starting epoch 4/10, LR = [0.01]
Step 150, Loss 0.12179103493690491
Starting epoch 5/10, LR = [0.01]
Starting epoch 6/10, LR = [0.01]
Step 200, Loss 0.01793655753135681
Starting epoch 7/10, LR = [0.01]
Step 250, Loss 0.00347137451171875
Starting epoch 8/10, LR = [0.01]
Step 300, Loss 0.003809235990047455
Starting epoch 9/10, LR = [0.01]


In [32]:
print(accuracies)

[0.951, 0.462, 0.31666666666666665, 0.21875, 0.1928, 0.15916666666666668, 0.1372857142857143, 0.118125, 0.10366666666666667, 0.0929]


**Define plot function**

In [0]:
def accuracy_plot(accuracies):
  from scipy import interpolate

  tck,u     = interpolate.splprep( [[i*10 for i in range(1,len(accuracies)+1)],accuracies] ,s = 0 )
  xnew,ynew = interpolate.splev( np.linspace( 0, 1, 100 ), tck,der = 0)

  fig, ax = plt.subplots(figsize=(10,9), facecolor='white')

  plt.rc('font', size=20)
  plt.plot( [i*10 for i in range(1,len(accuracies)+1)],accuracies,'.' , xnew ,ynew, label = "accuracy", c='orange' )
  ax.set_ylabel("Accuracy")
  ax.set_xlabel("Classes")
  plt.grid(axis='y')
  for in_i, in_j in zip([i*10 for i in range(1,len(accuracies)+1)], accuracies):  # Plot also the value of the point close to it
          ax.annotate(str(round(in_j, 3)), xy=(in_i, in_j))

  plt.savefig('test.png', format='png', dpi=300)
  plt.show()

**Plot accuracies**

In [0]:
accuracy_plot(accuracies)